In [1]:
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [2]:
n_termos = 59991

### Carregando modelo em disco ###
from sklearn.externals import joblib
joblib_file = "modelos/Supervisionado/encoder(50).pkl"
model_eforest = joblib.load(joblib_file)

In [3]:
k = 0 #Índice da árvore escolhida
estimator = model_eforest.estimators_[k]

In [4]:
n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold

# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes"
      % n_nodes)
r = ""
n_folhas = 0
for i in range(n_nodes):
    if is_leaves[i]:
        r += ("node=%s leaf node.\n" % (i))
        n_folhas += 1
    else:
        r += ("node=%s test node: go to node %s if X[:, %s] <= %ss else to "
              "node %s.\n"
              % (i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
        '''
        print("%snode=%s test node: go to node %s if X[:, %s] <= %ss else to "
              "node %s."
              % (int(node_depth[i]) * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
        '''
print('Leave: ' + str(n_folhas))

file = 'arvore.txt'
output = open(file, 'w+')
output.write(r)

The binary tree structure has 4147 nodes
Leave: 2074


202765

In [12]:
### Carregando coleção original ###
folder = "acmTesteTreino/"
arquivo = "treino0.txt"

file_c = folder + arquivo
file = open(file_c, 'r', encoding="utf8")
docs = file.readlines()
file.close()

n_termos = 59991

#Y (classes)
Y = []
for doc in docs:
    Y.append(doc.split()[0])
    
#X (features)
X = np.zeros((len(docs), n_termos), dtype=np.int_)

for index,doc in enumerate(docs):   
    for termo in doc.split()[1:]:
        X[index][int(termo.split(':')[0])] = int(termo.split(':')[1])
            
X_encode = model_eforest.encode(X)

node_samples = []

for t in range(len(X_encode[0])):
    node_samples.append({})
    for f in range(len(docs)):
        if X_encode[f][t] in node_samples[t].keys():
            node_samples[t][X_encode[f][t]] += 1
        else:
            node_samples[t][X_encode[f][t]] = 1

In [35]:
X_encode[0][0]

2860

In [31]:
altura_max = estimator.tree_.max_depth
altura_max

50

In [36]:
node_depth[X_encode[0][0]]

50.0

In [32]:
altura = altura_max - node_depth[X_encode[0][0]]
altura

0.0

In [34]:
alpha = 0.8
beta = 0.2
peso = alpha * (altura/(altura_max-1))
peso

0.0

In [ ]:
40.11346520735014